Libs

In [210]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re

Global variables

In [211]:
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
main_df = pd.DataFrame(columns = ["Stadium", "Capacity", "Spectators", "Average",	"Matches", "sold out", "Capacity", "Club", "Year"])
seasons = [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
dfs = []

Main function

In [212]:
def open_transfermarket_attendences():
  for i in range(len(seasons)):
    url = f'https://www.transfermarkt.com/saudi-pro-league/besucherzahlen/wettbewerb/SA1/saison_id/{seasons[i]}/plus/1'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    html = scrapingHTML(driver, i)
    print(f'Season {seasons[i]} - Scraping done!')

Function to get data with Selenium and BS4

In [213]:
def scrapingHTML(driver, i):
  try:
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, "sp_message_iframe_851946")))
    driver.find_element(By.XPATH,"//*[@id='notice']/div[3]/div[3]/button").click()
  except NoSuchElementException as exc:
    pass
  
  table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
  data = table.get_attribute('outerHTML')
  soup = BeautifulSoup(data, 'html.parser')
  html = soup.find(name='table')
  df = pd.read_html(str(html))[0]
  df["Club"] = ""
  df["Year"] = seasons[i]
  dfs.append(df)
  driver.quit()
  return html

Executation 

In [215]:
open_transfermarket_attendences()

Season 2007 - Scraping done!
Season 2008 - Scraping done!
Season 2009 - Scraping done!
Season 2010 - Scraping done!
Season 2011 - Scraping done!
Season 2012 - Scraping done!
Season 2013 - Scraping done!
Season 2014 - Scraping done!
Season 2015 - Scraping done!
Season 2016 - Scraping done!
Season 2017 - Scraping done!
Season 2018 - Scraping done!
Season 2019 - Scraping done!
Season 2020 - Scraping done!
Season 2021 - Scraping done!
Season 2022 - Scraping done!
Season 2023 - Scraping done!


Concat DataFrame list and create CSV

In [2]:
df_og = pd.concat(dfs)
df_og.to_csv("./RAW/ArabicSoccer/Attendances/Arabic_Attendances_2007_2023.csv", encoding='utf-8', index=False)

NameError: name 'pd' is not defined

Extract Teams info

In [250]:
arabic_leagues = ["SA1", "SA2L"]

for i in range(len(arabic_leagues)):
  url = f"https://www.transfermarkt.us/saudi-pro-league/startseite/wettbewerb/{arabic_leagues[i]}"
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)

  try:
      WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, "sp_message_iframe_851946")))
      driver.find_element(By.XPATH,"//*[@id='notice']/div[3]/div[3]/button").click()
  except NoSuchElementException as exc:
    pass

  table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
  data = table.get_attribute('outerHTML')
  soup = BeautifulSoup(data, 'html.parser')
  html = soup.find(name='table')
  df = pd.read_html(str(html))[0]
  df.to_csv(f"./RAW/ArabicSoccer/Teams/{arabic_leagues[i]}_teams_2023.csv", encoding='utf-8', index=False)

Extract All Champions - SPL

In [251]:
for i in range(len(arabic_leagues)):
  url = "https://www.transfermarkt.us/saudi-professional-league/erfolge/wettbewerb/SA1"
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(url)

  try:
      WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it(driver.find_element(By.ID, "sp_message_iframe_851946")))
      driver.find_element(By.XPATH,"//*[@id='notice']/div[3]/div[3]/button").click()
  except NoSuchElementException as exc:
    pass

  table = driver.find_element(By.XPATH, '//*[@id="yw1"]/table')
  data = table.get_attribute('outerHTML')
  soup = BeautifulSoup(data, 'html.parser')
  html = soup.find(name='table')
  df = pd.read_html(str(html))[0]
  df.to_csv(f"./RAW/ArabicSoccer/Champions/SA1_Champions.csv", encoding='utf-8', index=False)